In [1]:
import pandas as pd
import geopandas
import json
import os
from shapely.geometry import Point
import difflib
from geopy.distance import vincenty
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
inline_rc = dict(mpl.rcParams)

In [2]:
#in_folder = "audits/2018_07/data/"
out_folder = "audits/2018_11/images/"
in_folder = "../data/"
#out_folder = "/tmp"

# Analyse quantitative sur les routepoints

L'objectif est ici de déterminer combien de routepoints dans chaque jeu de données dispose d'informations sur l'accessibilité aux usagers de fauteuil roulant

## Données officielles

TODO : mettre un petit blabla pour expliquer les différentes valeurs possibles et sur quels objets on les trouve

In [3]:
#préparation géographique des données officielles pour les routepoints
gtfs_routepoints = pd.read_csv(os.path.join(in_folder,"opendata_routepoints.csv"), dtype=str)
for c in gtfs_routepoints.columns:
    new_col = "gtfs_" + c
    gtfs_routepoints.rename(columns={c: new_col}, inplace=True)

gtfs_routepoints = gtfs_routepoints.fillna("")

#gtfs_routepoints

In [4]:

def gtfs_wheelchair(row):
    if row["gtfs_route_wheelchair"] == "" or row["gtfs_stop_wheelchair"] == "":
        return "accessibilité inconnue"
    if row["gtfs_route_wheelchair"] == "1" and row["gtfs_stop_wheelchair"] == "1":
        return "accessible"
    if row["gtfs_route_wheelchair"] == "0" and row["gtfs_stop_wheelchair"] == "0":
        return "non accessible"
    else :
        return "partiellement accessible"

gtfs_routepoints['wheelchair_'] = gtfs_routepoints.apply(gtfs_wheelchair, axis=1)
gtfs_routepoints["wheelchair"] = gtfs_routepoints["wheelchair_"].astype("category")
gtfs_routepoints["wheelchair"] = gtfs_routepoints["wheelchair"].cat.set_categories(["accessibilité inconnue", "accessible", "partiellement accessible", "non accessible"])

print('sur un total de {} routepoints :'.format(len(gtfs_routepoints)))
gtfs_routepoints.groupby("wheelchair").size()

sur un total de 103423 routepoints :


wheelchair
accessibilité inconnue      69220
accessible                  25642
partiellement accessible     7622
non accessible                939
dtype: int64

## Données OpenStreetMap

TODO : idem, écrire un blabla pour expliquer les différentes valeurs possibles et sur quels objets on les trouve, et pourquoi on a choisi de garder juste stop et route.

In [5]:
#préparation géographique des données OSM pour les routepoints
osm_routepoints = pd.read_csv(os.path.join(in_folder, "osm_routepoints.csv"), dtype=str)
for c in osm_routepoints.columns:
    new_col = "osm_" + c
    osm_routepoints.rename(columns={c: new_col}, inplace=True)
    
osm_routepoints = osm_routepoints.fillna("")

In [6]:

def osm_wheelchair(row):
    if row["osm_stop_wheelchair"] == "" or row["osm_route_wheelchair"] == "":
        return "accessibilité inconnue"
    if row["osm_stop_wheelchair"] == "yes" and row["osm_route_wheelchair"] == "yes":
        return "accessible"
    if row["osm_stop_wheelchair"] == "no" and row["osm_route_wheelchair"] == "no":
        return "non accessible"
    else :
        return "partiellement accessible"

osm_routepoints['wheelchair_'] = osm_routepoints.apply(osm_wheelchair, axis=1)
osm_routepoints["wheelchair"] = osm_routepoints["wheelchair_"].astype("category")
osm_routepoints["wheelchair"] = osm_routepoints["wheelchair"].cat.set_categories(["accessibilité inconnue", "accessible", "partiellement accessible", "non accessible"])

print('sur un total de {} routepoints :'.format(len(osm_routepoints)))
osm_routepoints.groupby("wheelchair").size()

sur un total de 35382 routepoints :


wheelchair
accessibilité inconnue      33124
accessible                   1450
partiellement accessible      771
non accessible                 37
dtype: int64